In [29]:
from tensorflow import keras 
import os, shutil
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import models
from keras.layers import Dropout
from tensorflow.keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras 
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.test.is_built_with_cuda())
print(tf.version.VERSION)

df=pd.read_csv("C:/Users/Migue/Desktop/thesis_data/centralized_truth.csv")
tf.config.experimental.list_physical_devices()

Num GPUs Available:  1
True
2.7.0


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import pandas as pd

# create a dictionary to map column names to category names
category_map = {
    'MEL': 'MEL',
    'Benign': 'Benign',
    'Other Diseases': 'Other Diseases'
}

# create a list of category names based on the column values
categories = []
for index, row in df.iterrows():
    category = ''
    for column, value in row.iteritems():
        if value == 1:
            category += category_map[column] + ' '
    categories.append(category.strip())

# add the categories column to the DataFrame
df['category'] = pd.Series(categories)
df

C:\Users\Migue\AppData\Local\Temp\ipykernel_25900\613501458.py:14: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, value in row.iteritems():


image  MEL  Benign  Other Diseases category
0      ISIC_0000000.jpg    0       1               0   Benign
1      ISIC_0000001.jpg    0       1               0   Benign
2      ISIC_0000002.jpg    1       0               0      MEL
3      ISIC_0000003.jpg    0       1               0   Benign
4      ISIC_0000004.jpg    1       0               0      MEL
...                 ...  ...     ...             ...      ...
78278        458192.jpg    0       1               0   Benign
78279        497621.jpg    0       1               0   Benign
78280        499701.jpg    0       1               0   Benign
78281        515125.jpg    0       1               0   Benign
78282      587862-h.jpg    0       1               0   Benign

[78283 rows x 5 columns]

In [4]:
base_dir="C:/Users/Migue/Desktop/thesis_data/image_splits"
images_dir="C:/Users/Migue/Desktop/thesis_data/centralized_images"

train_dir = os.path.join(base_dir, 'Training')
test_dir = os.path.join(base_dir, 'Testing')
validate_dir = os.path.join(base_dir, 'validate')

for directory in [base_dir, train_dir, test_dir, validate_dir]:
    
    if not os.path.exists(directory):

        os.mkdir(directory)
        print(directory, "created")
        
subdirectories_MEL = []
subdirectories_Benign = []
subdirectories_Other_Diseases = []


for subdirectory in [train_dir, test_dir, validate_dir]:
    
    subdirectories_MEL.append(os.path.join(subdirectory, 'MEL'))
    if not os.path.exists(os.path.join(subdirectory, 'MEL')):
        os.mkdir(os.path.join(subdirectory, 'MEL'))
        print(os.path.join(subdirectory, 'MEL'), "created")
    
    subdirectories_Benign.append(os.path.join(subdirectory, 'Benign'))
    if not os.path.exists(os.path.join(subdirectory, 'Benign')):
        os.mkdir(os.path.join(subdirectory, 'Benign'))
        print(os.path.join(subdirectory, 'Benign'), "created")
        
    subdirectories_Other_Diseases.append(os.path.join(subdirectory, 'Other_Diseases'))
    if not os.path.exists(os.path.join(subdirectory, 'Other_Diseases')):
        os.mkdir(os.path.join(subdirectory, 'Other_Diseases'))
        print(os.path.join(subdirectory, 'Other_Diseases'), "created")



cross validation is disregarded, it is instead made a validation folder 

In [5]:

        
images = os.listdir(images_dir)
# taking 80% as train data 
x_train, x_test1, y_train, y_test1 = train_test_split(images, df["category"] ,test_size=0.2, random_state=42)

# from 20% data, splitting 50% as test data and 50% as validation data
x_test, x_val, y_test, y_val = train_test_split(x_test1, y_test1 ,test_size=0.5, random_state=42)

print('Training data set size :', y_train.shape[0])
print('Test data set size :', y_test.shape[0])
print('Validation data set size :', y_val.shape[0])


Training data set size : 62626
Test data set size : 7828
Validation data set size : 7829


In [23]:
x=0
for file in x_train:
    cat = df[df['image'] == file].iloc[0].category
    src_file = os.path.join('centralized_images/', file)
    dst_file = os.path.join(f'image_splits/Training/{cat}/', file)
    print(f"Sending file {src_file} to {dst_file}")
    x+=1
    shutil.copyfile(src_file, dst_file)
print(x)   

Sending file centralized_images/ISIC_0064362.jpg to image_splits/Training/Benign/ISIC_0064362.jpg
Sending file centralized_images/ISIC_0842426.jpg to image_splits/Training/Benign/ISIC_0842426.jpg
Sending file centralized_images/ISIC_4623037.jpg to image_splits/Training/Benign/ISIC_4623037.jpg
Sending file centralized_images/ISIC_0062840.jpg to image_splits/Training/Benign/ISIC_0062840.jpg
Sending file centralized_images/SSM17_orig.jpg to image_splits/Training/MEL/SSM17_orig.jpg
Sending file centralized_images/ISIC_0065489.jpg to image_splits/Training/Other Diseases/ISIC_0065489.jpg
Sending file centralized_images/ISIC_0024909.jpg to image_splits/Training/Benign/ISIC_0024909.jpg
Sending file centralized_images/ISIC_0029047.jpg to image_splits/Training/Benign/ISIC_0029047.jpg
Sending file centralized_images/ISIC_4217963.jpg to image_splits/Training/Benign/ISIC_4217963.jpg
Sending file centralized_images/ISIC_4750803.jpg to image_splits/Training/Benign/ISIC_4750803.jpg
Sending file centra

Sending file centralized_images/ISIC_0032687.jpg to image_splits/Training/MEL/ISIC_0032687.jpg
Sending file centralized_images/033356HB.jpg to image_splits/Training/Benign/033356HB.jpg
Sending file centralized_images/ISIC_0394140.jpg to image_splits/Training/Benign/ISIC_0394140.jpg
Sending file centralized_images/ISIC_6346989.jpg to image_splits/Training/Benign/ISIC_6346989.jpg
Sending file centralized_images/ISIC_0025469.jpg to image_splits/Training/Benign/ISIC_0025469.jpg
Sending file centralized_images/ISIC_0030228.jpg to image_splits/Training/Benign/ISIC_0030228.jpg
Sending file centralized_images/ISIC_1574020.jpg to image_splits/Training/Benign/ISIC_1574020.jpg
Sending file centralized_images/ISIC_0015179.jpg to image_splits/Training/Benign/ISIC_0015179.jpg
Sending file centralized_images/ISIC_4006060.jpg to image_splits/Training/Benign/ISIC_4006060.jpg
Sending file centralized_images/atypical-nevi-dermoscopy-96.jpg to image_splits/Training/Benign/atypical-nevi-dermoscopy-96.jpg
S

Sending file centralized_images/ISIC_0071398.jpg to image_splits/Training/Other Diseases/ISIC_0071398.jpg
Sending file centralized_images/ISIC_0059386.jpg to image_splits/Training/MEL/ISIC_0059386.jpg
Sending file centralized_images/011658HB.jpg to image_splits/Training/Benign/011658HB.jpg
Sending file centralized_images/ISIC_0033034.jpg to image_splits/Training/MEL/ISIC_0033034.jpg
Sending file centralized_images/024592VB.jpg to image_splits/Training/Benign/024592VB.jpg
Sending file centralized_images/Acl223.jpg to image_splits/Training/Benign/Acl223.jpg
Sending file centralized_images/ISIC_3596549.jpg to image_splits/Training/Benign/ISIC_3596549.jpg
Sending file centralized_images/ISIC_9535651.jpg to image_splits/Training/Benign/ISIC_9535651.jpg
Sending file centralized_images/ISIC_0061488.jpg to image_splits/Training/Benign/ISIC_0061488.jpg
Sending file centralized_images/actinic-keratosis-nose-2.jpg to image_splits/Training/Other Diseases/actinic-keratosis-nose-2.jpg
Sending file c

Sending file centralized_images/ISIC_0069863.jpg to image_splits/Training/Other Diseases/ISIC_0069863.jpg
Sending file centralized_images/010322HB.jpg to image_splits/Training/Benign/010322HB.jpg
Sending file centralized_images/ISIC_0600084.jpg to image_splits/Training/Benign/ISIC_0600084.jpg
Sending file centralized_images/ISIC_9514330.jpg to image_splits/Training/Benign/ISIC_9514330.jpg
Sending file centralized_images/9a83d251748a70f86a367787d3bb4c72.jpg to image_splits/Training/Benign/9a83d251748a70f86a367787d3bb4c72.jpg
Sending file centralized_images/ISIC_0013000.jpg to image_splits/Training/Benign/ISIC_0013000.jpg
Sending file centralized_images/a38959bd42ea324d64637f9f6dffc1d9.jpg to image_splits/Training/Benign/a38959bd42ea324d64637f9f6dffc1d9.jpg
Sending file centralized_images/85eecdee71e8ac56831eb6bf276c28bb.jpg to image_splits/Training/Other Diseases/85eecdee71e8ac56831eb6bf276c28bb.jpg
Sending file centralized_images/rhus-dermatitis-60.jpg to image_splits/Training/Benign/r

Sending file centralized_images/ISIC_5176708.jpg to image_splits/Training/Benign/ISIC_5176708.jpg
Sending file centralized_images/ISIC_0062015.jpg to image_splits/Training/Other Diseases/ISIC_0062015.jpg
Sending file centralized_images/actinic-keratosis-nose-7.jpg to image_splits/Training/Other Diseases/actinic-keratosis-nose-7.jpg
Sending file centralized_images/ISIC_3710757.jpg to image_splits/Training/Benign/ISIC_3710757.jpg
Sending file centralized_images/001812HB.jpg to image_splits/Training/Other Diseases/001812HB.jpg
Sending file centralized_images/ISIC_0463832.jpg to image_splits/Training/Benign/ISIC_0463832.jpg
Sending file centralized_images/ISIC_0029227.jpg to image_splits/Training/Benign/ISIC_0029227.jpg
Sending file centralized_images/ISIC_0028862.jpg to image_splits/Training/Benign/ISIC_0028862.jpg
Sending file centralized_images/600410cdc0fcd7be30fbf4f51f92b7cf.jpg to image_splits/Training/Benign/600410cdc0fcd7be30fbf4f51f92b7cf.jpg
Sending file centralized_images/ISIC_5

...

Sending file centralized_images/ISIC_6872938.jpg to image_splits/Training/Benign/ISIC_6872938.jpg
Sending file centralized_images/ISIC_5219334.jpg to image_splits/Training/Benign/ISIC_5219334.jpg
Sending file centralized_images/ISIC_1654183.jpg to image_splits/Training/Benign/ISIC_1654183.jpg
Sending file centralized_images/ISIC_0033778.jpg to image_splits/Training/Benign/ISIC_0033778.jpg
Sending file centralized_images/ISIC_4247778.jpg to image_splits/Training/Benign/ISIC_4247778.jpg
Sending file centralized_images/ISIC_2516636.jpg to image_splits/Training/Benign/ISIC_2516636.jpg
Sending file centralized_images/ISIC_0030464.jpg to image_splits/Training/Benign/ISIC_0030464.jpg
Sending file centralized_images/021119HB.jpg to image_splits/Training/Benign/021119HB.jpg
Sending file centralized_images/Dermis Malignant (469).jpg to image_splits/Training/MEL/Dermis Malignant (469).jpg
Sending file centralized_images/202c0e3a6dfe43805c43b8be44d3bacf.jpg to image_splits/Training/Benign/202c0e3a

Sending file centralized_images/ISIC_7998282.jpg to image_splits/Training/Benign/ISIC_7998282.jpg
Sending file centralized_images/033562VB.jpg to image_splits/Training/Benign/033562VB.jpg
Sending file centralized_images/perioral-dermatitis-98.jpg to image_splits/Training/Benign/perioral-dermatitis-98.jpg
Sending file centralized_images/005581HB.jpg to image_splits/Training/Benign/005581HB.jpg
Sending file centralized_images/ISIC_8076729.jpg to image_splits/Training/Benign/ISIC_8076729.jpg
Sending file centralized_images/04ff8956172642998a8edbcaa0960f09.jpg to image_splits/Training/Benign/04ff8956172642998a8edbcaa0960f09.jpg
Sending file centralized_images/55f29d5e1fc765bc2f169bcce8578469.jpg to image_splits/Training/MEL/55f29d5e1fc765bc2f169bcce8578469.jpg
Sending file centralized_images/ISIC_0032393.jpg to image_splits/Training/Benign/ISIC_0032393.jpg
Sending file centralized_images/ISIC_2212352.jpg to image_splits/Training/Benign/ISIC_2212352.jpg
Sending file centralized_images/ISIC_

Sending file centralized_images/ISIC_5380326.jpg to image_splits/Training/Benign/ISIC_5380326.jpg
Sending file centralized_images/ISIC_0032656.jpg to image_splits/Training/MEL/ISIC_0032656.jpg
Sending file centralized_images/ISIC_0014628.jpg to image_splits/Training/Benign/ISIC_0014628.jpg
Sending file centralized_images/ISIC_0053601.jpg to image_splits/Training/Other Diseases/ISIC_0053601.jpg
Sending file centralized_images/004311HB.jpg to image_splits/Training/Other Diseases/004311HB.jpg
Sending file centralized_images/Nil076.jpg to image_splits/Training/MEL/Nil076.jpg
Sending file centralized_images/Dermis Malignant (475).jpg to image_splits/Training/MEL/Dermis Malignant (475).jpg
Sending file centralized_images/ISIC_0973194.jpg to image_splits/Training/Benign/ISIC_0973194.jpg
Sending file centralized_images/e912bad5716487bf8367b8f80f490c8c.jpg to image_splits/Training/Benign/e912bad5716487bf8367b8f80f490c8c.jpg
Sending file centralized_images/ISIC_0033107.jpg to image_splits/Traini

Sending file centralized_images/ISIC_0071700.jpg to image_splits/Training/MEL/ISIC_0071700.jpg
Sending file centralized_images/ISIC_1602440.jpg to image_splits/Training/Benign/ISIC_1602440.jpg
Sending file centralized_images/ISIC_0028740.jpg to image_splits/Training/Benign/ISIC_0028740.jpg
Sending file centralized_images/ISIC_7729357.jpg to image_splits/Training/Benign/ISIC_7729357.jpg
Sending file centralized_images/008203HB.jpg to image_splits/Training/Benign/008203HB.jpg
Sending file centralized_images/ISIC_1642492.jpg to image_splits/Training/Benign/ISIC_1642492.jpg
Sending file centralized_images/ISIC_0071072.jpg to image_splits/Training/Benign/ISIC_0071072.jpg
Sending file centralized_images/ISIC_6163859.jpg to image_splits/Training/Benign/ISIC_6163859.jpg
Sending file centralized_images/ISIC_0072717.jpg to image_splits/Training/Benign/ISIC_0072717.jpg
Sending file centralized_images/ISIC_0025100.jpg to image_splits/Training/Benign/ISIC_0025100.jpg
Sending file centralized_images

Sending file centralized_images/ISIC_7013549.jpg to image_splits/Training/Benign/ISIC_7013549.jpg
Sending file centralized_images/ISIC_4007215.jpg to image_splits/Training/Benign/ISIC_4007215.jpg
Sending file centralized_images/ISIC_7198492.jpg to image_splits/Training/Benign/ISIC_7198492.jpg
Sending file centralized_images/ISIC_0060273.jpg to image_splits/Training/Other Diseases/ISIC_0060273.jpg
Sending file centralized_images/ISIC_6653225.jpg to image_splits/Training/Benign/ISIC_6653225.jpg
Sending file centralized_images/ISIC_4690506.jpg to image_splits/Training/Benign/ISIC_4690506.jpg
Sending file centralized_images/008285HB.jpg to image_splits/Training/Benign/008285HB.jpg
Sending file centralized_images/Gal191.jpg to image_splits/Training/Benign/Gal191.jpg
Sending file centralized_images/ISIC_0028459.jpg to image_splits/Training/Benign/ISIC_0028459.jpg
Sending file centralized_images/ISIC_8569148.jpg to image_splits/Training/Benign/ISIC_8569148.jpg
Sending file centralized_images/

In [24]:
x=0
for file in x_val:    
    cat = df[df['image'] == file].iloc[0].category
    src_file = os.path.join('centralized_images/', file)
    dst_file = os.path.join(f'image_splits/validate/{cat}/', file)
    print(f"Sending file {src_file} to {dst_file}")
    x+=1
    shutil.copyfile(src_file, dst_file)
print(x) 

Sending file centralized_images/ISIC_2682598.jpg to image_splits/validate/Benign/ISIC_2682598.jpg
Sending file centralized_images/ISIC_8157972.jpg to image_splits/validate/Benign/ISIC_8157972.jpg
Sending file centralized_images/ISIC_0059436.jpg to image_splits/validate/Benign/ISIC_0059436.jpg
Sending file centralized_images/ISIC_8699105.jpg to image_splits/validate/Benign/ISIC_8699105.jpg
Sending file centralized_images/ISIC_0026386.jpg to image_splits/validate/Benign/ISIC_0026386.jpg
Sending file centralized_images/ISIC_4871616.jpg to image_splits/validate/Benign/ISIC_4871616.jpg
Sending file centralized_images/ISIC_0559566.jpg to image_splits/validate/Benign/ISIC_0559566.jpg
Sending file centralized_images/ISIC_0157923.jpg to image_splits/validate/Benign/ISIC_0157923.jpg
Sending file centralized_images/ISIC_5078356.jpg to image_splits/validate/Benign/ISIC_5078356.jpg
Sending file centralized_images/ISIC_0013244.jpg to image_splits/validate/Benign/ISIC_0013244.jpg
Sending file central

Sending file centralized_images/ISIC_9375930.jpg to image_splits/validate/Benign/ISIC_9375930.jpg
Sending file centralized_images/ISIC_5170323.jpg to image_splits/validate/Benign/ISIC_5170323.jpg
Sending file centralized_images/ISIC_0030534.jpg to image_splits/validate/Benign/ISIC_0030534.jpg
Sending file centralized_images/ISIC_5153725.jpg to image_splits/validate/Benign/ISIC_5153725.jpg
Sending file centralized_images/ISIC_7145266.jpg to image_splits/validate/Benign/ISIC_7145266.jpg
Sending file centralized_images/ISIC_6378798.jpg to image_splits/validate/Benign/ISIC_6378798.jpg
Sending file centralized_images/027720VB.jpg to image_splits/validate/Benign/027720VB.jpg
Sending file centralized_images/ISIC_0024481.jpg to image_splits/validate/MEL/ISIC_0024481.jpg
Sending file centralized_images/d7b1a5eaa39e62d280cc64bc5f966bb3.jpg to image_splits/validate/Benign/d7b1a5eaa39e62d280cc64bc5f966bb3.jpg
Sending file centralized_images/ISIC_0817569.jpg to image_splits/validate/Benign/ISIC_081

Sending file centralized_images/ISIC_0071618.jpg to image_splits/validate/Benign/ISIC_0071618.jpg
Sending file centralized_images/ISIC_3926305.jpg to image_splits/validate/Benign/ISIC_3926305.jpg
Sending file centralized_images/ISIC_3173007.jpg to image_splits/validate/Benign/ISIC_3173007.jpg
Sending file centralized_images/ISIC_1654357.jpg to image_splits/validate/Benign/ISIC_1654357.jpg
Sending file centralized_images/ISIC_4751661.jpg to image_splits/validate/Benign/ISIC_4751661.jpg
Sending file centralized_images/basal-cell-carcinoma-lesion-3.jpg to image_splits/validate/Other Diseases/basal-cell-carcinoma-lesion-3.jpg
Sending file centralized_images/988caf18c233893f4e0b378af98a5f5f.jpg to image_splits/validate/Benign/988caf18c233893f4e0b378af98a5f5f.jpg
Sending file centralized_images/c74cfff2d234cd251f5b4099e0582159.jpg to image_splits/validate/Benign/c74cfff2d234cd251f5b4099e0582159.jpg
Sending file centralized_images/ISIC_0060036.jpg to image_splits/validate/Benign/ISIC_0060036.

Sending file centralized_images/ISIC_0064099.jpg to image_splits/validate/Other Diseases/ISIC_0064099.jpg
Sending file centralized_images/ISIC_3428742.jpg to image_splits/validate/Benign/ISIC_3428742.jpg
Sending file centralized_images/ISIC_3401780.jpg to image_splits/validate/Benign/ISIC_3401780.jpg
Sending file centralized_images/ISIC_0026255.jpg to image_splits/validate/Benign/ISIC_0026255.jpg
Sending file centralized_images/ISIC_2099059.jpg to image_splits/validate/Benign/ISIC_2099059.jpg
Sending file centralized_images/ISIC_6580109.jpg to image_splits/validate/Benign/ISIC_6580109.jpg
Sending file centralized_images/ISIC_0013813.jpg to image_splits/validate/MEL/ISIC_0013813.jpg
Sending file centralized_images/atypical-nevi-dermoscopy-67.jpg to image_splits/validate/Benign/atypical-nevi-dermoscopy-67.jpg
Sending file centralized_images/ISIC_8223466.jpg to image_splits/validate/Benign/ISIC_8223466.jpg
Sending file centralized_images/ISIC_0188926.jpg to image_splits/validate/Benign/IS

Sending file centralized_images/ISIC_0063383.jpg to image_splits/validate/Benign/ISIC_0063383.jpg
Sending file centralized_images/ISIC_4544729.jpg to image_splits/validate/Benign/ISIC_4544729.jpg
Sending file centralized_images/013564HB.jpg to image_splits/validate/Benign/013564HB.jpg
Sending file centralized_images/ISIC_0029781.jpg to image_splits/validate/Other Diseases/ISIC_0029781.jpg
Sending file centralized_images/ISIC_4964532.jpg to image_splits/validate/Benign/ISIC_4964532.jpg
Sending file centralized_images/ISIC_0030921.jpg to image_splits/validate/Benign/ISIC_0030921.jpg
Sending file centralized_images/ISIC_1897327.jpg to image_splits/validate/Benign/ISIC_1897327.jpg
Sending file centralized_images/ISIC_0064007.jpg to image_splits/validate/Benign/ISIC_0064007.jpg
Sending file centralized_images/ISIC_0029575.jpg to image_splits/validate/MEL/ISIC_0029575.jpg
Sending file centralized_images/ISIC_0062899.jpg to image_splits/validate/Other Diseases/ISIC_0062899.jpg
Sending file ce

...

Sending file centralized_images/ISIC_4311436.jpg to image_splits/validate/Benign/ISIC_4311436.jpg
Sending file centralized_images/ISIC_0031481.jpg to image_splits/validate/Benign/ISIC_0031481.jpg
Sending file centralized_images/ISIC_0026835.jpg to image_splits/validate/Benign/ISIC_0026835.jpg
Sending file centralized_images/ISIC_7746409.jpg to image_splits/validate/Benign/ISIC_7746409.jpg
Sending file centralized_images/023001HB.jpg to image_splits/validate/Benign/023001HB.jpg
Sending file centralized_images/ISIC_0025276.jpg to image_splits/validate/Benign/ISIC_0025276.jpg
Sending file centralized_images/ISIC_7399786.jpg to image_splits/validate/Benign/ISIC_7399786.jpg
Sending file centralized_images/ISIC_4257537.jpg to image_splits/validate/Benign/ISIC_4257537.jpg
Sending file centralized_images/ISIC_0058489.jpg to image_splits/validate/Benign/ISIC_0058489.jpg
Sending file centralized_images/ISIC_0063221.jpg to image_splits/validate/Benign/ISIC_0063221.jpg
Sending file centralized_ima

Sending file centralized_images/ISIC_0057687.jpg to image_splits/validate/MEL/ISIC_0057687.jpg
Sending file centralized_images/a1b3c793674c7fec9ff9c8ca1a019dd1.jpg to image_splits/validate/Benign/a1b3c793674c7fec9ff9c8ca1a019dd1.jpg
Sending file centralized_images/ISIC_0065013.jpg to image_splits/validate/MEL/ISIC_0065013.jpg
Sending file centralized_images/968e1a5756f64d872387e90182a2babf.jpg to image_splits/validate/Benign/968e1a5756f64d872387e90182a2babf.jpg
Sending file centralized_images/ISIC_8633889.jpg to image_splits/validate/Benign/ISIC_8633889.jpg
Sending file centralized_images/ISIC_5728988.jpg to image_splits/validate/Benign/ISIC_5728988.jpg
Sending file centralized_images/nevus-spilus-8.jpg to image_splits/validate/Benign/nevus-spilus-8.jpg
Sending file centralized_images/ISIC_9934401.jpg to image_splits/validate/Benign/ISIC_9934401.jpg
Sending file centralized_images/ISIC_1630301.jpg to image_splits/validate/MEL/ISIC_1630301.jpg
Sending file centralized_images/1ee7c2a9070

Sending file centralized_images/ISIC_1788746.jpg to image_splits/validate/Benign/ISIC_1788746.jpg
Sending file centralized_images/ISIC_0028342.jpg to image_splits/validate/Benign/ISIC_0028342.jpg
Sending file centralized_images/ISIC_0062294.jpg to image_splits/validate/Benign/ISIC_0062294.jpg
Sending file centralized_images/Nml020.jpg to image_splits/validate/MEL/Nml020.jpg
Sending file centralized_images/ISIC_0799535.jpg to image_splits/validate/Benign/ISIC_0799535.jpg
Sending file centralized_images/ISIC_0507787.jpg to image_splits/validate/Benign/ISIC_0507787.jpg
Sending file centralized_images/ISIC_6931573.jpg to image_splits/validate/Benign/ISIC_6931573.jpg
Sending file centralized_images/ISIC_0072761.jpg to image_splits/validate/Other Diseases/ISIC_0072761.jpg
Sending file centralized_images/ISIC_0068363.jpg to image_splits/validate/Other Diseases/ISIC_0068363.jpg
Sending file centralized_images/005802HB.jpg to image_splits/validate/Benign/005802HB.jpg
Sending file centralized_im

Sending file centralized_images/ISIC_6388815.jpg to image_splits/validate/Benign/ISIC_6388815.jpg
Sending file centralized_images/ISIC_0065223.jpg to image_splits/validate/Benign/ISIC_0065223.jpg
Sending file centralized_images/ISIC_0030459.jpg to image_splits/validate/Benign/ISIC_0030459.jpg
Sending file centralized_images/ISIC_0033474.jpg to image_splits/validate/Benign/ISIC_0033474.jpg
Sending file centralized_images/ISIC_9199393.jpg to image_splits/validate/Benign/ISIC_9199393.jpg
Sending file centralized_images/ISIC_0026217.jpg to image_splits/validate/Benign/ISIC_0026217.jpg
Sending file centralized_images/ISIC_7595859.jpg to image_splits/validate/Benign/ISIC_7595859.jpg
Sending file centralized_images/ISIC_0065514.jpg to image_splits/validate/Benign/ISIC_0065514.jpg
Sending file centralized_images/003320HB.jpg to image_splits/validate/Other Diseases/003320HB.jpg
Sending file centralized_images/ISIC_0031393.jpg to image_splits/validate/Benign/ISIC_0031393.jpg
Sending file central

Sending file centralized_images/ISIC_5176105.jpg to image_splits/validate/Benign/ISIC_5176105.jpg
Sending file centralized_images/ISIC_0032150.jpg to image_splits/validate/Benign/ISIC_0032150.jpg
Sending file centralized_images/nevus-anemicus-12.jpg to image_splits/validate/Benign/nevus-anemicus-12.jpg
Sending file centralized_images/ISIC_8430286.jpg to image_splits/validate/Benign/ISIC_8430286.jpg
Sending file centralized_images/ISIC_5329156.jpg to image_splits/validate/Benign/ISIC_5329156.jpg
Sending file centralized_images/ISIC_0014974.jpg to image_splits/validate/Benign/ISIC_0014974.jpg
Sending file centralized_images/011158HB.jpg to image_splits/validate/Benign/011158HB.jpg
Sending file centralized_images/010127HB.jpg to image_splits/validate/Benign/010127HB.jpg
Sending file centralized_images/ISIC_0056263.jpg to image_splits/validate/MEL/ISIC_0056263.jpg
Sending file centralized_images/001322HB.jpg to image_splits/validate/Other Diseases/001322HB.jpg
Sending file centralized_imag

In [27]:
x=0
for file in x_test:
    cat = df[df['image'] == file].iloc[0].category
    src_file = os.path.join('centralized_images/', file)
    dst_file = os.path.join(f'image_splits/Testing/{cat}/', file)
    print(f"Sending file {src_file} to {dst_file}")
    x+=1
    shutil.copyfile(src_file, dst_file)
print(x) 

Sending file centralized_images/ISIC_2811896.jpg to image_splits/Testing/Benign/ISIC_2811896.jpg
Sending file centralized_images/ISIC_6692344.jpg to image_splits/Testing/MEL/ISIC_6692344.jpg
Sending file centralized_images/ISIC_8595133.jpg to image_splits/Testing/Benign/ISIC_8595133.jpg
Sending file centralized_images/ISIC_0057803.jpg to image_splits/Testing/Benign/ISIC_0057803.jpg
Sending file centralized_images/ISIC_0053790.jpg to image_splits/Testing/MEL/ISIC_0053790.jpg
Sending file centralized_images/47d2cf8ad5e2a8f673060480a1c3462f.jpg to image_splits/Testing/Other Diseases/47d2cf8ad5e2a8f673060480a1c3462f.jpg
Sending file centralized_images/000529HB.jpg to image_splits/Testing/Other Diseases/000529HB.jpg
Sending file centralized_images/ISIC_1251811.jpg to image_splits/Testing/Benign/ISIC_1251811.jpg
Sending file centralized_images/ISIC_0070904.jpg to image_splits/Testing/Other Diseases/ISIC_0070904.jpg
Sending file centralized_images/ISIC_0000134.jpg to image_splits/Testing/Beni

Sending file centralized_images/ISIC_0032018.jpg to image_splits/Testing/Benign/ISIC_0032018.jpg
Sending file centralized_images/ISIC_0030821.jpg to image_splits/Testing/Other Diseases/ISIC_0030821.jpg
Sending file centralized_images/ISIC_0056100.jpg to image_splits/Testing/Other Diseases/ISIC_0056100.jpg
Sending file centralized_images/ISIC_9090822.jpg to image_splits/Testing/Benign/ISIC_9090822.jpg
Sending file centralized_images/ISIC_9595560.jpg to image_splits/Testing/Benign/ISIC_9595560.jpg
Sending file centralized_images/ISIC_0026509.jpg to image_splits/Testing/Benign/ISIC_0026509.jpg
Sending file centralized_images/ISIC_8225551.jpg to image_splits/Testing/Benign/ISIC_8225551.jpg
Sending file centralized_images/ISIC_8013225.jpg to image_splits/Testing/Benign/ISIC_8013225.jpg
Sending file centralized_images/ISIC_8092875.jpg to image_splits/Testing/Benign/ISIC_8092875.jpg
Sending file centralized_images/ISIC_0033286.jpg to image_splits/Testing/MEL/ISIC_0033286.jpg
Sending file cent

Sending file centralized_images/ISIC_3686329.jpg to image_splits/Testing/Benign/ISIC_3686329.jpg
Sending file centralized_images/ISIC_7220622.jpg to image_splits/Testing/Benign/ISIC_7220622.jpg
Sending file centralized_images/ISIC_3624807.jpg to image_splits/Testing/Benign/ISIC_3624807.jpg
Sending file centralized_images/ISIC_0029801.jpg to image_splits/Testing/Benign/ISIC_0029801.jpg
Sending file centralized_images/a2f2e0b2fa53e22c965dfeb876d96189.jpg to image_splits/Testing/Benign/a2f2e0b2fa53e22c965dfeb876d96189.jpg
Sending file centralized_images/ISIC_8606969.jpg to image_splits/Testing/Benign/ISIC_8606969.jpg
Sending file centralized_images/ISIC_5029679.jpg to image_splits/Testing/Benign/ISIC_5029679.jpg
Sending file centralized_images/ISIC_2421486.jpg to image_splits/Testing/Benign/ISIC_2421486.jpg
Sending file centralized_images/ISIC_0029596.jpg to image_splits/Testing/Benign/ISIC_0029596.jpg
Sending file centralized_images/ISIC_1892011.jpg to image_splits/Testing/Benign/ISIC_18

Sending file centralized_images/ISIC_0056309.jpg to image_splits/Testing/MEL/ISIC_0056309.jpg
Sending file centralized_images/ISIC_3766837.jpg to image_splits/Testing/Benign/ISIC_3766837.jpg
Sending file centralized_images/ISIC_8617277.jpg to image_splits/Testing/MEL/ISIC_8617277.jpg
Sending file centralized_images/ISIC_7998509.jpg to image_splits/Testing/Benign/ISIC_7998509.jpg
Sending file centralized_images/004399HB.jpg to image_splits/Testing/Other Diseases/004399HB.jpg
Sending file centralized_images/ccac742910bc9649b0651d9fec0aedba.jpg to image_splits/Testing/Other Diseases/ccac742910bc9649b0651d9fec0aedba.jpg
Sending file centralized_images/ISIC_8441293.jpg to image_splits/Testing/Benign/ISIC_8441293.jpg
Sending file centralized_images/032216VB.jpg to image_splits/Testing/Benign/032216VB.jpg
Sending file centralized_images/epidermal-nevus-48.jpg to image_splits/Testing/Benign/epidermal-nevus-48.jpg
Sending file centralized_images/83ee5dc2e7d2cb7f20dede25203883a3.jpg to image_spl

Sending file centralized_images/ISIC_0006800.jpg to image_splits/Testing/Benign/ISIC_0006800.jpg
Sending file centralized_images/acne-keloidalis-43.jpg to image_splits/Testing/Benign/acne-keloidalis-43.jpg
Sending file centralized_images/ISIC_0059474.jpg to image_splits/Testing/Other Diseases/ISIC_0059474.jpg
Sending file centralized_images/ISIC_0029135.jpg to image_splits/Testing/Benign/ISIC_0029135.jpg
Sending file centralized_images/ISIC_1007014.jpg to image_splits/Testing/Benign/ISIC_1007014.jpg
Sending file centralized_images/perioral-dermatitis-59.jpg to image_splits/Testing/Benign/perioral-dermatitis-59.jpg
Sending file centralized_images/008829VB.jpg to image_splits/Testing/Benign/008829VB.jpg
Sending file centralized_images/Nil031.jpg to image_splits/Testing/MEL/Nil031.jpg
Sending file centralized_images/ISIC_6332165.jpg to image_splits/Testing/Benign/ISIC_6332165.jpg
Sending file centralized_images/ISIC_0025202.jpg to image_splits/Testing/Benign/ISIC_0025202.jpg
Sending file 

...

Sending file centralized_images/ISIC_3348306.jpg to image_splits/Testing/Benign/ISIC_3348306.jpg
Sending file centralized_images/ISIC_0059342.jpg to image_splits/Testing/Other Diseases/ISIC_0059342.jpg
Sending file centralized_images/ISIC_0711360.jpg to image_splits/Testing/Benign/ISIC_0711360.jpg
Sending file centralized_images/ISIC_0032958.jpg to image_splits/Testing/MEL/ISIC_0032958.jpg
Sending file centralized_images/ISIC_0030072.jpg to image_splits/Testing/Benign/ISIC_0030072.jpg
Sending file centralized_images/ISIC_0024584.jpg to image_splits/Testing/Benign/ISIC_0024584.jpg
Sending file centralized_images/ISIC_0032897.jpg to image_splits/Testing/Other Diseases/ISIC_0032897.jpg
Sending file centralized_images/ISIC_0056772.jpg to image_splits/Testing/Benign/ISIC_0056772.jpg
Sending file centralized_images/stasis-dermatitis-177.jpg to image_splits/Testing/Benign/stasis-dermatitis-177.jpg
Sending file centralized_images/ISIC_0032002.jpg to image_splits/Testing/Benign/ISIC_0032002.jpg

Sending file centralized_images/ISIC_0027456.jpg to image_splits/Testing/Benign/ISIC_0027456.jpg
Sending file centralized_images/ISIC_0062698.jpg to image_splits/Testing/Benign/ISIC_0062698.jpg
Sending file centralized_images/717f9cab8254cf44e153be0794f8e710.jpg to image_splits/Testing/Benign/717f9cab8254cf44e153be0794f8e710.jpg
Sending file centralized_images/ISIC_6225557.jpg to image_splits/Testing/Benign/ISIC_6225557.jpg
Sending file centralized_images/033076HB.jpg to image_splits/Testing/Benign/033076HB.jpg
Sending file centralized_images/ISIC_2461940.jpg to image_splits/Testing/Benign/ISIC_2461940.jpg
Sending file centralized_images/ISIC_0013643.jpg to image_splits/Testing/Benign/ISIC_0013643.jpg
Sending file centralized_images/ISIC_0008552.jpg to image_splits/Testing/Benign/ISIC_0008552.jpg
Sending file centralized_images/ISIC_0071108.jpg to image_splits/Testing/Other Diseases/ISIC_0071108.jpg
Sending file centralized_images/ISIC_5673518.jpg to image_splits/Testing/MEL/ISIC_56735

Sending file centralized_images/rosacea-98.jpg to image_splits/Testing/Benign/rosacea-98.jpg
Sending file centralized_images/019312HB.jpg to image_splits/Testing/Other Diseases/019312HB.jpg
Sending file centralized_images/045137HB.jpg to image_splits/Testing/Benign/045137HB.jpg
Sending file centralized_images/ISIC_5252704.jpg to image_splits/Testing/Benign/ISIC_5252704.jpg
Sending file centralized_images/056a9a4fca15486be30ca4c845526b6b.jpg to image_splits/Testing/Other Diseases/056a9a4fca15486be30ca4c845526b6b.jpg
Sending file centralized_images/ISIC_3205354.jpg to image_splits/Testing/Benign/ISIC_3205354.jpg
Sending file centralized_images/ISIC_0034243.jpg to image_splits/Testing/MEL/ISIC_0034243.jpg
Sending file centralized_images/ISIC_0010853.jpg to image_splits/Testing/Benign/ISIC_0010853.jpg
Sending file centralized_images/ISIC_8669067.jpg to image_splits/Testing/Benign/ISIC_8669067.jpg
Sending file centralized_images/ISIC_6768044.jpg to image_splits/Testing/Benign/ISIC_6768044.j

Sending file centralized_images/ISIC_0000545.jpg to image_splits/Testing/Benign/ISIC_0000545.jpg
Sending file centralized_images/ISIC_0025080.jpg to image_splits/Testing/Benign/ISIC_0025080.jpg
Sending file centralized_images/ISIC_0002476.jpg to image_splits/Testing/Benign/ISIC_0002476.jpg
Sending file centralized_images/ISIC_3261511.jpg to image_splits/Testing/Benign/ISIC_3261511.jpg
Sending file centralized_images/3875c968bfb44ad137180f3b944132c2.jpg to image_splits/Testing/Other Diseases/3875c968bfb44ad137180f3b944132c2.jpg
Sending file centralized_images/ISIC_4648604.jpg to image_splits/Testing/Benign/ISIC_4648604.jpg
Sending file centralized_images/012663HB.jpg to image_splits/Testing/Benign/012663HB.jpg
Sending file centralized_images/10549-6.jpg to image_splits/Testing/Other Diseases/10549-6.jpg
Sending file centralized_images/ISIC_0014557.jpg to image_splits/Testing/Benign/ISIC_0014557.jpg
Sending file centralized_images/ISIC_6087560.jpg to image_splits/Testing/Benign/ISIC_6087

Sending file centralized_images/e923714f9e9f1efa1b8ba453445e699c.jpg to image_splits/Testing/Other Diseases/e923714f9e9f1efa1b8ba453445e699c.jpg
Sending file centralized_images/ISIC_4553390.jpg to image_splits/Testing/Benign/ISIC_4553390.jpg
Sending file centralized_images/ISIC_0010009.jpg to image_splits/Testing/Benign/ISIC_0010009.jpg
Sending file centralized_images/ISIC_7281080.jpg to image_splits/Testing/Benign/ISIC_7281080.jpg
Sending file centralized_images/ISIC_0031039.jpg to image_splits/Testing/Benign/ISIC_0031039.jpg
Sending file centralized_images/012911VB.jpg to image_splits/Testing/Benign/012911VB.jpg
Sending file centralized_images/26ad865bcdc627dfa0dc2e419252a0ed.jpg to image_splits/Testing/MEL/26ad865bcdc627dfa0dc2e419252a0ed.jpg
Sending file centralized_images/ISIC_0032985.jpg to image_splits/Testing/MEL/ISIC_0032985.jpg
Sending file centralized_images/Dermis Benign  (97).jpg to image_splits/Testing/Benign/Dermis Benign  (97).jpg
Sending file centralized_images/019564H

In [30]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=(0.95, 0.95),
    rotation_range=15,
    horizontal_flip=True,
    vertical_flip=True,
    data_format="channels_last",
    dtype=tf.float32
)

test_datagen = ImageDataGenerator(rescale=1./255)

img_size=256

trainGen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=3,
    class_mode='categorical')

valGen = test_datagen.flow_from_directory(
    validate_dir,
    target_size=(img_size, img_size),
    batch_size=1,
    class_mode='categorical')

testGen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=1,
    class_mode='categorical')

Found 62626 images belonging to 3 classes.
Found 7828 images belonging to 3 classes.
Found 7828 images belonging to 3 classes.
